### Importar bibliotecas

In [93]:
from Grafo import *
import numpy as np
# import o alfabeto em codigo Ascii 
import string 
# manipular os dados de excel com xlrd
import xlrd

## Alfabeto Σ e Estados


In [94]:
tamanhoAlfabeto =len(string.printable)
print(len(string.printable))
alfabeto = []

def criaAlfabeto():#funcao para criar o alfabeto
    alfabeto.extend(string.printable)  #cria uma nova lista contendo o alfabeto

criaAlfabeto()
print (alfabeto)

# Ajuste do numero de estados para criar o automato
numEstados = 10
estadosList = [i for i in range(numEstados)]
estadosList

100
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r', '\x0b', '\x0c']


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [115]:
try:
    # import banco de dados palabras reservadas (excel.xls) 
    excel = xlrd.open_workbook('C:/Users/gaboh/Documents/GitHub/BNF/Tabela-Simbolos2.xls') #import xlrd
    sh = excel.sheet_by_index(0) #Folha 1 ou sheet 1
except:
    print("Exception ->Tabela de simbolos tem que ser um doc Excel do tipo .xls")

tabelaSimbolos ={} #dictionary
numColumnas = sh.nrows-2
for key in range(numColumnas):
    cellCadeia = sh.cell(key+1,1).value
    cellCategoria = sh.cell(key+1,2).value
    tabelaSimbolos[cellCadeia] = cellCategoria
# Tabela Simbolos -> colocar as pabras dentro de uma tabela hash ( Em python o dictionary representa a tabela hash )
print(tabelaSimbolos)
# busca na tabela de simbolos se existe a palavra 
tabelaSimbolos["For"]

{'While': 'TK_While', 'If': 'TK_If', 'For': 'TK_For', 'Main': 'TK_Main', 'DoWhile': 'TK_Do_While', 'Fun': 'TK_Fun', '+': 'TK_Soma', '-': 'TK_Subracao', '*': 'TK_Multiplicacao', '/': 'TK_Divisao', ';': 'TK_Final_Lexema', 'x': 'TK_Identificador'}


'TK_For'

### Construcao do Automato 

In [96]:
def main():
    p = 'x'
    grafo = Grafo()
    for x in range(numEstados): agregar(grafo, estadosList[x])

    relacionar(grafo, estadosList[0], estadosList[1], p)
    relacionar(grafo, estadosList[2], estadosList[4], p)
    relacionar(grafo, estadosList[3], estadosList[5], p)
    relacionar(grafo, estadosList[4], estadosList[8], p)
    relacionar(grafo, estadosList[5], estadosList[7], p)
    relacionar(grafo, estadosList[6], estadosList[5], p)
    relacionar(grafo, estadosList[8], estadosList[2], p)
    relacionar(grafo, estadosList[7], estadosList[1], p)
    relacionar(grafo, estadosList[4], estadosList[2], p)
    relacionar(grafo, estadosList[2], estadosList[3], p)
    relacionar(grafo, estadosList[1], estadosList[6], p)

    imprimeGrafo(grafo)


In [97]:
if __name__ == "__main__":
    main()

{0: [->1 peso=x], 1: [->6 peso=x], 2: [->4 peso=x, ->3 peso=x], 3: [->5 peso=x], 4: [->8 peso=x, ->2 peso=x], 5: [->7 peso=x], 6: [->5 peso=x], 7: [->1 peso=x], 8: [->2 peso=x], 9: []}
